In [ ]:
%%capture
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install "torch>=2.1.1" -U

In [ ]:
!pip install  accelerate

In [ ]:
!pip install -q python-dotenv==1.0.0

!pip install -q pandas==1.5.3
!pip install -q unstructured==0.7.12
!pip install -q wikipedia==1.4.0
!pip install -q pypdf==3.12.0
!pip install -q jq==1.4.1
!pip install -q nltk==3.8.1
!pip install -q tiktoken==0.4.0
!pip install -q  sentencepiece==0.1.99
!pip install -q sentence-transformers==2.2.2
!pip install -q cohere==4.11.2
!pip install -q faiss-cpu==1.7.4
!pip install -q chromadb==0.3.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.2/421.2 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.9 

In [ ]:
!pip install transformers


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
import re
from sklearn.metrics.pairwise import cosine_similarity

Load the model

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
load_in_8bit=True  # Enable 8-bit quantization for model loading
bnb_8bit_quant_type="...your preferred 8-bit quantization type..."  # Specify the desired 8-bit quantization type (e.g., "pact", "ttq")
bnb_8bit_compute_dtype="uint8"  # Set the compute data type to 8-bit unsigned integer (uint8)
bnb_8bit_use_double_quant=False
# Activate nested quantization for 8-bit base models
use_nested_quant = True
device_map = {"": 0}
compute_dtype = getattr(torch, bnb_8bit_compute_dtype)

bnb_config =BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit quantization for model loading
    bnb_8bit_quant_type="...",  # Specify your preferred 8-bit quantization type (e.g., "pact", "ttq")
    bnb_8bit_compute_dtype="uint8",  # Set compute data type to 8-bit unsigned integer
    bnb_8bit_use_double_quant=False,  # Disable nested quantization (if not applicable)
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and load_in_8bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Unused kwargs: ['bnb_8bit_quant_type', 'bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
argilla 1.28.0 requires typer<0.10.0,>=0.6.0, but you have typer 0.12.3 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but y

In [ ]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.0 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
#LOAD DATA FOR RAG
llm_loader = PyPDFLoader("/content/A propos de Djezzy (24).pdf")
pages = llm_loader.load_and_split()

In [ ]:
len(pages)

37

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
import nltk
nltk.download('punkt')

text_splitter = CharacterTextSplitter(
    separator="separatoor",
    is_separator_regex=False,
    # Utilisez le séparateur de votre choix
    chunk_size=200,
    chunk_overlap=0
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Lire le contenu du fichier texte
with open('/content/Nouveau document texte (2).txt', 'r') as fichier:
    contenu = fichier.read()

# Séparer les mots en utilisant la virgule comme séparateur
mots = contenu.split(',')

# Afficher les mots pour vérifier
for mot in mots:
    print(mot.strip())

# Si vous souhaitez stocker les mots dans un tableau (une liste)
tableau_de_mots = [mot.strip() for mot in mots]

# Afficher le tableau de mots
print(tableau_de_mots)


A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
scoop
E facture
verso+
parental control
english learning
sms 3lik
callback relay
B2mail
tranquilo
double call
bank transfer
roaming
golden
personal account
*777#
scoop ramadan
ranati ramadan
nokta
VRxperience
anghami
youscribe
premium gaming
clip+
red list
conference call
international service
E flexy
recharge djezzy
flexy
flexily
legend
zid
3ayla
comfort partage
comfort
Maxi
special
bezzaf
izzy
tunisia
hadj
internet access
BIP SMS
phishing
['A propos de djezzy', 'A propos de djezzy', 'A propos de djezzy', 'A propos de djezzy', 'A propos de djezzy', 'A propos de djezzy', 'A propos de djezzy', '

In [ ]:
len(tableau_de_mots)

63

In [ ]:
documents=["".join([pages[i].page_content for i in range(len(pages))])]
metadatas = [ {"document":mot.strip() }for mot in mots  ]

In [ ]:
print(metadatas)

[{'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'A propos de djezzy'}, {'document': 'scoop'}, {'document': 'E facture'}, {'document': 'verso+'}, {'document': 'parental control'}, {'document': 'english learning'}, {'document': 'sms 3lik'}, {'document': 'callback relay'}, {'document': 'B2mail'}, {'document': 'tranq

In [ ]:
tokens_chunks = text_splitter.create_documents(
    documents, metadatas=metadatas
)

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
docs_text = [ chunk.page_content for chunk in tokens_chunks ]
docs_embeddings = embedding_llm.embed_documents(docs_text)

In [ ]:
from langchain.vectorstores import FAISS
vector_db = FAISS.from_documents(tokens_chunks, embedding_llm)

In [ ]:
len(docs_text)

65

In [ ]:
i=0
similar_docs=[]
for i in range(len(mots_trouves)):
  k=mots_trouves[i]
  result=vector_db.similarity_search(
                                      query_text,
                                      k=1,
                                      filter={'document':mots_trouves[i] }
                                    )
  similar_docs.append(result[0])
  context="\n---------------------\n".join([similar_docs[i].page_content for i in range(len(similar_docs))])


In [ ]:
similar_docs[0].metadata

{'document': 'zid'}

In [ ]:
import pandas as pd
chemin_fichier = "data_test.txt"
questions = []
reponses = []
with open(chemin_fichier, "r") as fichier:
    lines = fichier.readlines()
    lines = [line.strip() for line in lines if line.strip()]
    lines = [line.replace('"', '').replace("'", '').replace('\\', ''.replace(",", ' ')) for line in lines]
    i = 0
    while i < len(lines):
            # Extract the question and the corresponding answer.
            question = lines[i].strip()
            reponse = lines[i+1].strip()

            questions.append(question)
            reponses.append(reponse)
            # Move to the next question-answer pair.
            i += 2
print(len(questions))
print(len(reponses))

292
292


Test the model with the test questions.

In [ ]:
import numpy as np
i=0
mots_a_verifier = tableau_de_mots
generates=[]
for i in range(151):
 query_text = questions[i]
 query_embedding = embedding_llm.embed_query(query_text)
 query_embedding_array = np.array(query_embedding)
 docs_embeddings=np.array(docs_embeddings)



  # Question to analyze.
 question = query_text

  # Convert the question into a list of words.
 mots_question = question.lower().split()
 bi_grammes = [' '.join([mots_question[i], mots_question[i+1]]) for i in range(len(mots_question)-1)]

 mots_a_verifier_lower = {mot.lower(): mot for mot in mots_a_verifier}
 mots_question_lower=[mot.lower() for mot in mots_question]
 bi_grammes_lower=[mot.lower() for mot in bi_grammes]
  #Find the words in the question that are in the table.
 mots_trouves1 = [mots_a_verifier_lower[mot] for mot in mots_a_verifier_lower if  mot in bi_grammes_lower]

 if not mots_trouves1:
  mots_trouves1 = [mots_a_verifier_lower[mot] for mot in mots_a_verifier_lower if mot in mots_question_lower ]
# Display the found words.

 mots_trouves=mots_trouves1
 if not mots_trouves:

  similarities = [cosine_similarity(doc.reshape(1,-1), query_embedding_array.reshape(1,-1)) for doc in docs_embeddings]
  sorted_docs = sorted(zip(docs_text, docs_embeddings, similarities), key=lambda x: x[2], reverse=True)
  similar_docs1 = [(doc,sim) for doc, _, sim in sorted_docs if sim > 0.72]
  if  not similar_docs1:
   similar_docs2 = [(doc,sim) for doc, _, sim in sorted_docs if sim > 0.65]
   if  not similar_docs2:
    similar_docs = [(doc,sim) for doc, _, sim in sorted_docs if sim > 0.4]
    if  not similar_docs:
      similar_docsA = [(doc,sim) for doc, _, sim in sorted_docs if (sim >= 0.3 and sim<0.4)]
      if  not similar_docsA:
        prompt = f"[INST] <<SYS>>\n  <</SYS>>\n\n  ###question: {query_text} [/INST]"
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,temperature=0.1,top_p=0.9, max_length=200)
        result = pipe(prompt)
        repons=result[0]['generated_text'].split('[/INST]')[1].strip()
        generate=repons.replace("<start_of_turn>model", "")
        generates.append(generate)
        print(generate)
      else:
       prompt = f"[INST] <<SYS>>\n  <</SYS>>\n\n  ###question: {query_text} [/INST]"
       pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,temperature=0.1,top_p=0.9, max_length=200)
       result = pipe(prompt)
       repons=result[0]['generated_text'].split('[/INST]')[1].strip()
       generate=repons.replace("<start_of_turn>model", "")
       generates.append(generate)
       print(generate)
    else:
       context="\n---------------------\n".join([doc for doc,_ in similar_docs[:4]]if len(similar_docs) >=3 else [doc for doc, _ in similar_docs[:1]])
       system_message=" "
       prompt = f"[INST] <<SYS>>\n <</SYS>>\n\n  ###question: {query_text} [/INST]"
       #prompt = f" <bos><start_of_turn>user \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[0]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[0]}<eos>\nuser \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[1]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[1]}<eos>\nuser read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{context}\n###question:\n{query_text}\n###answer:\n<end_of_turn>\n <start_of_turn>model" # replace the command here with something relevant to your task
       pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,temperature=0.1,top_p=0.9, max_length=200)
       result = pipe(prompt)
       repons=result[0]['generated_text'].split('[/INST]')[1].strip()
       generate=repons.replace("<start_of_turn>model", "")
       generates.append(generate)
       print(generate)
 
   else:
    context = "\n---------------------\n".join([doc for doc, _ in similar_docs2[:2]] if len(similar_docs2) >= 2 else [doc for doc, _ in similar_docs2[:1]])
    system_message=" "
    prompt = f"[INST] <<SYS>>\n  <</SYS>>\n\n  ###question: {query_text} [/INST]"
    #prompt = f" <bos><start_of_turn>user \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[0]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[0]}<eos>\nuser \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[1]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[1]}<eos>\nuser read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{context}\n###question:\n{query_text}\n###answer:\n<end_of_turn>\n <start_of_turn>model" # replace the command here with something relevant to your task
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,temperature=0.1,top_p=0.9, max_length=200)
    result = pipe(prompt)
    repons=result[0]['generated_text'].split('[/INST]')[1].strip()
    generate=repons.replace("<start_of_turn>model", "")
    generates.append(generate)
    print(generate)


  else:
   context="\n---------------------\n".join([doc for doc,_ in similar_docs1[:1]])
   system_message=" "
   prompt = f"[INST] <<SYS>> <</SYS>>\n\n {query_text}[/INST]"
   #prompt = f" <bos><start_of_turn>user \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[0]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[0]}<eos>\nuser \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[1]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[1]}<eos>\nuser read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{context}\n###question:\n{query_text}\n###answer:\n<end_of_turn>\n <start_of_turn>model" # replace the command here with something relevant to your task
   pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,temperature=0.1,top_p=0.9, max_length=200)
   result = pipe(prompt)
   repons=result[0]['generated_text'].split('[/INST]')[1].strip()
   generate=repons.replace("<start_of_turn>model", "")
   generates.append(generate)
   print(generate)
   #print(result[0]['generated_text'])
 else:
    i=0
    similar_docs=[]
    for i in range(len(mots_trouves)):
      k=mots_trouves[i]
      result=vector_db.similarity_search(
                                      query_text,
                                      k=1,
                                      filter={'document':mots_trouves[i] }
                                    )
      similar_docs.append(result[0])
    context="\n---------------------\n".join([similar_docs[i].page_content for i in range(len(similar_docs))])
    system_message=" "
    prompt = f"[INST] <<SYS>> <</SYS>>\n\n  ###question: {query_text} [/INST]"
    #prompt = f" <bos><start_of_turn>user \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[0]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[0]}<eos>\nuser \n read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{contexts[1]}\n ###question:\nWhat are the benefits of opting for the Djezzy Legend 100 DA package? \n###answer:\n{reponses[1]}<eos>\nuser read each paraphrase in the context and Answer  the  question .\ndo not take into consideration the paragraphs which have no relation to the question\n if there is not a paragraph that is related to the question, respond that for this question it's best to reach out to our customer service team . They'll be able to assist you with your needs\n just give me the answer I don't want any other details \n###context:\n{context}\n###question:\n{query_text}\n###answer:\n<end_of_turn>\n <start_of_turn>model" # replace the command here with something relevant to your task
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,temperature=0.1,top_p=0.9, max_length=200)
    result = pipe(prompt)
    repons=result[0]['generated_text'].split('[/INST]')[1].strip()
    generate=repons.replace("<start_of_turn>model", "")
    generates.append(generate)
    print(generate)
    #print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The Legend 2500 DA package is a comprehensive and powerful solution for data acquisition and analysis in various fields, including scientific research, industrial automation, and medical applications. Here are some of the key benefits of the Legend 2500 DA package:

1. High-speed data acquisition: The Legend 2500 DA can acquire data at extremely high speeds, making it ideal for applications that require real-time data acquisition and analysis.
2. High-resolution signal processing: The package includes advanced signal processing capabilities, allowing for the analysis of high-resolution signals with high accuracy and precision.
3. Flexible data acquisition: The Legend 2500 DA can be used to
I'm not able to provide information on the Djezzy Legend 1 500 DA or any other product as I'm just an AI and do not have access to real-time information or specific details about products. Additionally, I cannot provide personal opinions or recommendations.

However, I can suggest some general benefi

KeyboardInterrupt: 

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.3 MB/s eta 0:00:00


Blue metric

In [ ]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU()
s=0
for i in range(44):
 hypothesis = generates[i]
 reference = reponses[i]

 score = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference],
 )


 print(score.score/100)
 s=s+score.score/100
moyen=s/44
print(f"the blue moyen is {moyen}")

0.009375923147696174
0.026417552343499327
0.010220344546720288
0.014006453316529334
0.012782381249289643
0.005527099121042454
0.005858518567668101
0.009958590547628147
0.029310905545522035
0.00523905205729294
0.008001280438458866
0.005112089427054479
0.014856594307610865
0.0045462776386880534
0.009750974603211324
0.005285625988830918
0.012993731884458697
0.017915952897736082
0.017101038979839524
0.004934681772447047
0.013136313846844057
0.0051307007703583814
0.016596702394213108
0.004699659932552321
0.03702111218901429
0.00913289617808974
0.004791055297479523
0.008855637992702413
0.008889369856244776
0.011985162958990101
0.009475478661764186
0.052592180355742826
0.08381971618643787


0.018028298453547382
0.007923404808221247
0.013565590497345195
0.017666093699802387
0.009708217543484625
0.00378998288567802
0.020269923170348023
0.004340393455341845
0.005464888870526168
0.005258074194598571
0.13520459769143478
the blue moyen is 0.016739557278908777


rouge metric


In [ ]:
pip install rouge

In [ ]:
from rouge import Rouge
rouge_scorer = Rouge()
s=0
i=0
for i in range(44):
  print(i)

  hypothesis = generates[i]
  reference = reponses[i]

  score1 = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
  )
  print(score1[0]["rouge-l"]["f"])
  s=s+score1[0]["rouge-l"]["f"]
  moyen=s/44
print(f"the rouge moyen is {moyen}")

0
0.11111110730617296
1
0.12765957046627446
2
0.13592232642096344
3
0.12820512415516122
4
0.18181817836139175
5
0.08421052264598354
6
0.09411764346020776
7
0.17391304007561445
8
0.2168674670750472
9
0.09195401984410104
10
0.06349205955152455
11
0.10989010685182957
12
0.18367346479591848
13
0.07339449241646338
14
0.06593406303586535
15
0.10416666324869804
16
0.1555555520987655
17
0.13043477946124774
18
0.10869564902646513
19
0.10869564902646513
20
0.10958903738037168
21
0.07619047256235845
22
0.08791208460813923
23
0.04938271352537736
24
0.18604650818550572
25
0.07317072778108291
26
0.024999997450000262
27
0.07692307327108007
28
0.13861385766101372
29
0.11538461173261846
30
0.1359223263229335
31
0.24999999559911248
32
0.28571428118284053
33
0.16393442206933634
34
0.0983606519470574
35
0.18803418326247368
36
0.1714285675174604
37
0.11111110718278479
38
0.05063290937349791
39
0.04081632213244529
40
0.14814814598384393
41
0.08219177826233823
42
0.03333333055555578
43
0.399999995088889
the 

bert

In [ ]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 663.9 kB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer
# Example texts
p_moyen=0
f_moyen=0
R_moyen=0
for i in range(44):
 candidate= generates[i]
 reference = reponses[i]
# BERTScore calculation
 scorer = BERTScorer(model_type='bert-base-uncased')
 P, R, F1 = scorer.score([candidate], [reference])
 p_moyen=p_moyen+P
 f_moyen=f_moyen+F1
 R_moyen=R_moyen+R
 print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")
moyen_p=p_moyen/44
moyen_f=f_moyen/44
moyen_R=R_moyen/44

BERTScore Precision: 0.3808, Recall: 0.4515, F1: 0.4131
BERTScore Precision: 0.4408, Recall: 0.5178, F1: 0.4762
BERTScore Precision: 0.4391, Recall: 0.5422, F1: 0.4852
BERTScore Precision: 0.3985, Recall: 0.5421, F1: 0.4593
BERTScore Precision: 0.5061, Recall: 0.6250, F1: 0.5593
BERTScore Precision: 0.4553, Recall: 0.5473, F1: 0.4971
BERTScore Precision: 0.4241, Recall: 0.5765, F1: 0.4887
BERTScore Precision: 0.4677, Recall: 0.5796, F1: 0.5177
BERTScore Precision: 0.4307, Recall: 0.6594, F1: 0.5211
BERTScore Precision: 0.4098, Recall: 0.6133, F1: 0.4913
BERTScore Precision: 0.4129, Recall: 0.4654, F1: 0.4375
BERTScore Precision: 0.3755, Recall: 0.4672, F1: 0.4164
BERTScore Precision: 0.4551, Recall: 0.5116, F1: 0.4817
BERTScore Precision: 0.3777, Recall: 0.4416, F1: 0.4071
BERTScore Precision: 0.3724, Recall: 0.5381, F1: 0.4402
BERTScore Precision: 0.3606, Recall: 0.4494, F1: 0.4002
BERTScore Precision: 0.4539, Recall: 0.5999, F1: 0.5168
BERTScore Precision: 0.4116, Recall: 0.6009, F1:

In [ ]:
print(moyen_p)
print(moyen_f)
print(moyen_R)

tensor([0.4300])
tensor([0.4788])
tensor([0.5428])
